In [ ]:
import os
from glob import glob
import shutil
from zipfile import ZipFile
import json
from tqdm import tqdm
import pandas as pd
src = "/home/kai/workspace/DeepDocs_Project/datalake/source/tech_sci_mrc"

In [ ]:
data = glob(f"{src}/**/*.zip", recursive=True)

In [ ]:
# unzip to "./"
for file in data:
    print(f"Unzipping {file}...")
    with ZipFile(file, 'r') as zip_ref:
        zip_ref.extractall(os.path.dirname(file).replace(src, "./"))
    print(f"Unzipped {file} to {os.path.dirname(file)}")

In [ ]:
json_files = glob("152.기술과학_문서_기계독해_데이터/**/*.json", recursive=True)

In [ ]:
from bs4 import BeautifulSoup
def extract_tables_with_thead(html):
    soup = BeautifulSoup(html, "html.parser")
    tables = soup.find_all("table")
    results = []
    for table in tables:
        results.append(str(table))
    return results

In [ ]:
records = []
for json_path in tqdm(json_files):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    for context_info in data['dataset']['context_info']:
        context = context_info['context']
        tables = extract_tables_with_thead(context)
        if not tables:
            continue
        for table in tables:
            record = {
                "html": table,
            }
            records.append(record)
df = pd.DataFrame(records)

In [ ]:
df_unique = df.drop_duplicates(subset=['html'])
print(f"Total unique HTML tables: {len(df_unique)}")
print(f"Total HTML tables: {len(df)}")

In [ ]:
df_unique.to_parquet("tech_sci_mrc_tables.parquet", index=False)

In [ ]:
import asyncio
from playwright.async_api import async_playwright
async def render_html_and_get_bboxes(html, tags=("table", "tr", "td", "p", "h1", "h2")):
    async with async_playwright() as p:
        if not isinstance(html, str):
            html = str(html)
        browser = await p.chromium.launch()
        page = await browser.new_page(viewport={"width": 1280, "height": 1280})
        await page.set_content(html)
        await page.evaluate("""
            () => {
                // 기본 스타일 강제 적용
                let style = document.createElement('style');
                style.textContent = `
                    table { border-collapse: collapse; }
                    table, th, td { border: 1px solid #333; }
                    th, td { padding: 4px; }
                `;
                document.head.appendChild(style);
            }
        """)
        await page.wait_for_timeout(100)  # 렌더링 안정화
        bboxes = await page.evaluate(
            """
            (tags) => {
                let results = [];
                tags.forEach(tag => {
                    document.querySelectorAll(tag).forEach((el, idx) => {
                        let rect = el.getBoundingClientRect();
                        results.push({
                            tag: tag,
                            idx: idx,
                            text: el.innerText,
                            x: rect.left + window.scrollX,
                            y: rect.top + window.scrollY,
                            width: rect.width,
                            height: rect.height
                        });
                    });
                });
                return results;
            }
            """,
            list(tags)
        )
        img_bytes = await page.screenshot(full_page=True, type='jpeg')
        await browser.close()
        return img_bytes, bboxes

In [ ]:
tables[0]

In [ ]:
img_bytes, bboxes = await render_html_and_get_bboxes(tables[0])

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import io
img = Image.open(io.BytesIO(img_bytes))
draw = ImageDraw.Draw(img)
colors = [f"#{i:02x}{j:02x}00" for i in range(0, 256, 51) for j in range(0, 256, 51)]
for bbox in bboxes:
    tag = bbox['tag']
    text = bbox['text']
    x = bbox['x']
    y = bbox['y']
    width = bbox['width']
    height = bbox['height']
    # random outline color
    color = colors[bbox['idx'] % len(colors)]
    draw.rectangle([x, y, x + width, y + height], outline=color, width=3)

In [ ]:
img